In [1]:
import pandas as pd
import ast

**Dataset**

In [2]:
df = pd.read_csv("homework_10_exercise_4_results.csv")
df

,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
0,0,0.8579,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,[],1657243049
1,1,0.9474,0.8065,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,[],1657243049
2,2,0.8895,0.8065,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,arm,[],1657243049
3,3,0.8789,0.8065,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,arm,[],1657243049
4,4,0.8316,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.010,Adam,0.3,3,random,arm,[],1657243049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,5,0.6421,0.4839,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657302359
206,6,0.9105,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657302359
207,7,0.8579,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657302359
208,8,0.8632,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657302359


**The results, for each set of hyperparameters when run in the several environments.**

The results on any given platform are consistent when the GPU is not enabled.  When the GPU is enabled, results tend to vary.

In [3]:
for index in df.panel_index.unique():
    display(df[df.panel_index == index])

,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
0,0,0.8579,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,[],1657243049
10,0,0.9316,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243411
20,0,0.9053,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243578
30,0,0.9053,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243740
40,0,0.9105,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243913
50,0,0.9368,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657244076
60,0,0.8947,0.8710,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245418
70,0,0.9053,0.9032,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245473
80,0,0.8684,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245500
90,0,0.9368,0.9032,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245532


,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
1,1,0.9474,0.8065,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,[],1657243049
11,1,0.9895,0.8065,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243411
21,1,0.9789,0.7742,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243578
31,1,0.9842,0.7742,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243740
41,1,0.9526,0.8065,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243913
51,1,0.9632,0.7419,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657244076
61,1,0.9211,0.8065,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245418
71,1,0.9368,0.8387,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245473
81,1,0.9368,0.9032,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245500
91,1,0.9211,0.8065,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245532


,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
2,2,0.8895,0.8065,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,arm,[],1657243049
12,2,0.9105,0.7419,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243411
22,2,0.9158,0.7419,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243578
32,2,0.9158,0.8065,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243740
42,2,0.9316,0.7742,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243913
52,2,0.9053,0.7742,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657244076
62,2,0.8632,0.8065,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245418
72,2,0.8737,0.8387,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245473
82,2,0.8632,0.8387,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245500
92,2,0.8737,0.8387,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245532


,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
3,3,0.8789,0.8065,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,arm,[],1657243049
13,3,0.8947,0.8065,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243411
23,3,0.8947,0.8065,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243578
33,3,0.9000,0.8387,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243740
43,3,0.9053,0.8065,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243913
53,3,0.8842,0.8065,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657244076
63,3,0.8947,0.8065,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245418
73,3,0.9053,0.8065,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245473
83,3,0.9105,0.8387,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245500
93,3,0.8947,0.8065,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245532


,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
4,4,0.8316,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.01,Adam,0.3,3,random,arm,[],1657243049
14,4,0.8632,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.01,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243411
24,4,0.8632,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.01,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243578
34,4,0.8526,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.01,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243740
44,4,0.8474,0.8710,"(5, 5)","(1, 1)","(2, 2)",0.01,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243913
54,4,0.8526,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.01,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657244076
64,4,0.8368,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.01,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245418
74,4,0.7474,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.01,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245473
84,4,0.8526,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.01,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245500
94,4,0.7684,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.01,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245532


,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
5,5,0.7211,0.5806,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,arm,[],1657243049
15,5,0.6684,0.5806,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243411
25,5,0.6737,0.5806,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243578
35,5,0.6684,0.5806,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243740
45,5,0.6632,0.5806,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243913
55,5,0.6737,0.5806,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657244076
65,5,0.6421,0.4839,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245418
75,5,0.6421,0.4839,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245473
85,5,0.6421,0.4839,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245500
95,5,0.6421,0.4839,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245532


,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
6,6,0.9000,0.8710,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,arm,[],1657243049
16,6,0.9737,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243411
26,6,0.9895,0.8710,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243578
36,6,0.9105,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243740
46,6,0.9474,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243913
56,6,0.9368,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657244076
66,6,0.9421,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245418
76,6,0.9053,0.7419,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245473
86,6,0.9105,0.7419,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245500
96,6,0.8789,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245532


,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
7,7,0.8684,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,arm,[],1657243049
17,7,0.9211,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243411
27,7,0.8842,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243578
37,7,0.9211,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243740
47,7,0.8947,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243913
57,7,0.9053,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657244076
67,7,0.8789,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245418
77,7,0.8632,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245473
87,7,0.8316,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245500
97,7,0.8842,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245532


,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
8,8,0.9211,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,arm,[],1657243049
18,8,0.9263,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243411
28,8,0.9158,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243578
38,8,0.9263,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243740
48,8,0.9158,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243913
58,8,0.9105,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657244076
68,8,0.8211,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245418
78,8,0.8474,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245473
88,8,0.8421,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245500
98,8,0.8368,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245532


,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
9,9,0.9316,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,arm,[],1657243049
19,9,0.9000,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243411
29,9,0.9474,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243578
39,9,0.9158,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243740
49,9,0.8895,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657243913
59,9,0.9263,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657244076
69,9,0.8421,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245418
79,9,0.8421,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245473
89,9,0.8684,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245500
99,9,0.8526,0.8710,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245532


**All results, sort by validation accuracy**

In [4]:
df.groupby('panel_index', as_index=False)['validation_accuracy'].mean().sort_values('validation_accuracy', ascending=False)

,panel_index,validation_accuracy
0,0,0.846405
9,9,0.824919
3,3,0.824905
2,2,0.817205
4,4,0.814152
1,1,0.814143
7,7,0.801881
6,6,0.792638
8,8,0.789581
5,5,0.511529


In [5]:
df.sort_values('validation_accuracy', ascending=False)[0:20]

,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
120,0,0.9053,0.9355,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657280509
109,9,0.8895,0.9355,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245561
121,1,0.9316,0.9032,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657280509
81,1,0.9368,0.9032,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245500
70,0,0.9053,0.9032,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245473
150,0,0.9263,0.9032,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,[],1657299370
170,0,0.9474,0.9032,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657300192
90,0,0.9368,0.9032,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657245532
161,1,0.9316,0.9032,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657299965
134,4,0.8368,0.9032,"(5, 5)","(1, 1)","(2, 2)",0.010,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657297403


In [6]:
df[df.processor=='arm']

,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
0,0,0.8579,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,[],1657243049
1,1,0.9474,0.8065,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,[],1657243049
2,2,0.8895,0.8065,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,arm,[],1657243049
3,3,0.8789,0.8065,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,arm,[],1657243049
4,4,0.8316,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.010,Adam,0.3,3,random,arm,[],1657243049
5,5,0.7211,0.5806,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,arm,[],1657243049
6,6,0.9000,0.8710,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,arm,[],1657243049
7,7,0.8684,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,arm,[],1657243049
8,8,0.9211,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,arm,[],1657243049
9,9,0.9316,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,arm,[],1657243049


In [7]:
len(df[df.processor!='arm'])

150

In [8]:
df[df.gpu=="[]"]

,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
0,0,0.8579,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,[],1657243049
1,1,0.9474,0.8065,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,[],1657243049
2,2,0.8895,0.8065,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,arm,[],1657243049
3,3,0.8789,0.8065,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,arm,[],1657243049
4,4,0.8316,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.010,Adam,0.3,3,random,arm,[],1657243049
5,5,0.7211,0.5806,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,arm,[],1657243049
6,6,0.9000,0.8710,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,arm,[],1657243049
7,7,0.8684,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,arm,[],1657243049
8,8,0.9211,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,arm,[],1657243049
9,9,0.9316,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,arm,[],1657243049


In [9]:
len(df)

210